In [ ]:
# Importing necessary dependencies
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os
from IPython.display import clear_output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to the Images, (may need to change depending on where the Images and labels are stored)
path_image_train = "mapillary/train/image"
path_image_val = "mapillary/val/image"
path_image_test = "mapillary/test/image"

path_label_train = "mapillary/train/label"
path_label_val = "mapillary/val/label"

In [ ]:
cd drive/MyDrive/AI3/

/content/drive/MyDrive/AI3


In [ ]:
# 0: BACKGROUND 1: PEDESTRIAN 2: CAR

# Function that looks into the image and labels and converts them into a numpy array
# and holding them to a temporary variables
def appendNumpy(path,lists,temporary_variable,label,start,end):
    """
    Input informations:

        path: Path of where the Image is stored (String)
        
        lists: list of the directories in the path file (List)
        
        temporary_variables: temporary variable where the numpy array will be stored (Empty list)
        
        label: checking if the appending array is an Image or Label (Boolean)
        
        start: which part of the directory's index it's starting to save from (Int)
        
        end: which part of the directory's index it's save to (Int)
    
    """
    while(start<end):
        nparr = mpimg.imread(path+'/'+lists[start])
        if (label == True): # Applies only to the labels which it's data are adjusted due to ToTensor transformation

            # Segmentation preprocessing for Mapillary dataset
            arraysweepr = nparr[:,:,0]
            arraysweepg = nparr[:,:,1]
            arraysweepb = nparr[:,:,2]
            nparr=np.where((arraysweepr >= 0.8627451)&(arraysweepg <= 0.078431375)&(arraysweepb <= 0.23529412),1,np.where((arraysweepr == 0)&(arraysweepg == 0)&(arraysweepb == 0.5568628),2,0)) 

            # Segmentation preprocessing for BDD100k dataset
            # nparr=np.where((nparr > 0.04)&(nparr < 0.045),1,np.where((nparr > 0.05)&(nparr < 0.06),2,0)) # For BDD100k

            # Segmentation preprocessing for Cityscape dataset
            # nparr=np.where((nparr > 0.092)&(nparr < 0.095),1,np.where((nparr > 0.098)&(nparr < 0.11),2,0)) # For cityscape
            
            nparr=np.array(nparr,dtype='uint8')
        resized_nparr = cv2.resize(nparr, (320,320), interpolation = cv2.INTER_AREA)
        temporary_variable.append(resized_nparr)
        start=start+1
        print(start)

In [ ]:
# Function that saves sorts the Images and Labes to match and stores into array to be saved into 
# training, validation and testing dataset

def Imagesaver(mode):
    """
    mode: Boolean operation where it decides which one to store.
          0th index: store training data
          1st index: store validation data
          2nd index: store testing data
          
    example: [1,1,0] = store training and validation data
    
    Have to manually set where each image is saving from, and where until to below
    """
    # BDD100K: T=0-7000 V=0-1000
    # Cityscape: T=0-2975 V=0-500
    # M1: T=0-6000 V=0-667
    # M2: T=6000-12000 V=667-1333
    # M3: T=12000-18000 V=1333-2000
    training_start=12000
    validation_start=0
    testing_start=0
    
    training_end=18000
    validation_end=667
    testing_end=0
    
    # Initialisation for the temporary variables
    temptraimg = []
    temptralal = []
    tempvalimg = []
    tempvallal = []
    temptstimg = []
    
    # The listed sorted variables below sort the Images and Labels in the directory it is at
    # It is done as when listing the directory, they are not in order, and sorting them
    # in alphabetical order fixes this issue
    
    if (mode[0]==1):
        sortedtraimg = sorted(os.listdir(path_image_train))
        appendNumpy(path_image_train,sortedtraimg,temptraimg,False,training_start,training_end)

        sortedtralal = sorted(os.listdir(path_label_train))
        appendNumpy(path_label_train,sortedtralal,temptralal,True,training_start,training_end)
        
    if (mode[1]==1):
        sortedvalimg = sorted(os.listdir(path_image_val))
        appendNumpy(path_image_val,sortedvalimg,tempvalimg,False,validation_start,validation_end)
        
        sortedvallal = sorted(os.listdir(path_label_val))
        appendNumpy(path_label_val,sortedvallal,tempvallal,True,validation_start,validation_end)
        
    if (mode[2]==1):  
        sortedtstimg = sorted(os.listdir(path_image_test)) 
        appendNumpy(path_image_test,sortedtstimg,temptstimg,False,testing_start,testing_end)
        
    train_img=np.asarray(temptraimg)
    train_label=np.asarray(temptralal)
    validation_img=np.asarray(tempvalimg)
    validation_label=np.asarray(tempvallal)
    test_img=np.asarray(temptstimg)    
    print("----------------------Done----------------------")
    
    return train_img, train_label, validation_img, validation_label, test_img

In [ ]:
train_img, train_label, validation_img, validation_label, test_img = Imagesaver([0,0,0])

In [ ]:
img = validation_img[0]
lbl = validation_label[0]

plt.figure(figsize=(12,12))
imgplot = plt.imshow(img)
plt.figure(figsize=(12,12))
imgplot = plt.imshow(lbl)
plt.show()

In [ ]:
# Saves the temporary variable into numpy zip file
if (train_img.shape[0]>0 and train_label.shape[0]>0):
    np.savez("train.npz",train_img,train_label)
if (validation_img.shape[0]>0 and validation_label.shape[0]>0):
    np.savez("val.npz",validation_img,validation_label)
if (test_img.shape[0]>0):
    np.savez("test.npz",test_img)